## Author :- Jay Faldu
#### Email id :- jay.faldu928@gmail.com 

# ----------------------------------------------------------------------------------------
# Applied Data science Capstone Project
# ----------------------------------------------------------------------------------------

### Introduction of project:-

    This project is part of IBM Data Science Professional certification.

###     




### let's start the project!!!

In [1]:
!pip install geopy
!pip install lxml
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import HTML

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#  







# Load the Data from wikipedia (week :- 3 , part :- 1)

In [3]:
raw_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
data = pd.DataFrame(raw_data[0]).dropna(subset=['Neighborhood']).reset_index(drop=True)
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
data.shape

(103, 3)

#    






# (week :- 3 , part :- 2)

In [5]:
PC = pd.read_csv('https://cocl.us/Geospatial_data')
PC

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
longitude = list()

latitude=[]
longitude=[]

for pc in data['Postal Code']:
    i = PC.index[PC['Postal Code'] == pc].values
    latitude.append(PC.loc[i, 'Latitude'])
    longitude.append(PC.loc[i, 'Longitude'])

data['Latitude'] = latitude
data['Longitude'] = longitude

data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,"25 43.753259 Name: Latitude, dtype: float64","25 -79.329656 Name: Longitude, dtype: float64"
1,M4A,North York,Victoria Village,"34 43.725882 Name: Latitude, dtype: float64","34 -79.315572 Name: Longitude, dtype: float64"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","53 43.65426 Name: Latitude, dtype: float64","53 -79.360636 Name: Longitude, dtype: float64"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","71 43.718518 Name: Latitude, dtype: float64","71 -79.464763 Name: Longitude, dtype: float64"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","85 43.662301 Name: Latitude, dtype: float64","85 -79.389494 Name: Longitude, dtype: float64"


In [7]:
data.shape

(103, 5)

#    






# (week :- 3 , part :- 3)

In [8]:
Toronto_data = data[data['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(Toronto_data.shape)

(39, 5)


### information required to use Foursquare

In [9]:
client_ID = 'XCP1LBPKA54U331UGGORSK2K0QQUOWJQPTRQP0GNWC2GFSEC' 
client_secret = 'SDJEWGFFAYGXKXONJ0LZS1QTQ2TL4RS1SYONHILPKB2J2RLF' 
version = '20200416'
limit = 100
radius = 500

### extracts all the venues in the neighborhoods

In [10]:
def get_neighborhood_venues(names, latitudes, longitudes, limit=100, radius=500):
    venues_list = list()

    for name, lat, lng in zip(names, latitudes, longitudes):
        lat = lat.values[0]
        lng = lng.values[0]

        uri = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_ID}&client_secret={client_secret}' \
              f'&ll={lat},{lng}&v={version}&radius={radius}&limit={limit}'

        results = requests.get(uri).json()['response']['groups'][0]['items']

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results]
        )

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = [
            'Neighborhood',
            'Neighborhood Latitude',
            'Neighborhood Longitude',
            'Venue',
            'Venue Latitude',
            'Venue Longitude',
            'Venue Category'
        ]

    return nearby_venues

###  obtain all the venues

In [11]:
Toronto_venues = get_neighborhood_venues(Toronto_data['Neighborhood'], Toronto_data['Latitude'], Toronto_data['Longitude'])
print(Toronto_venues.shape)

(1612, 7)


###  preprocesses the data to prepare it for clustering

In [12]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

### most common venues

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
    
print(neighborhoods_venues_sorted.shape)

(39, 11)


### k-means clustering

In [14]:
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

### merges the data from Toronto to sort them based on neighborhood

In [15]:
Toronto_merged = Toronto_data

Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

### creates a map of Toronto with the associated 5 clusters previously found.¶

In [16]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
Toronto_latitude = location.latitude
Toronto_longitude = location.longitude

map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'],
                                  Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [17]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, '1st Most Common Venue']
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, '1st Most Common Venue']
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, '1st Most Common Venue']
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, '1st Most Common Venue']
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, '1st Most Common Venue']

19    Pool
Name: 1st Most Common Venue, dtype: object

In [18]:
Toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront","53 43.65426 Name: Latitude, dtype: float64","53 -79.360636 Name: Longitude, dtype: float64",0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","85 43.662301 Name: Latitude, dtype: float64","85 -79.389494 Name: Longitude, dtype: float64",0,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,College Auditorium,College Cafeteria
2,M5B,Downtown Toronto,"Garden District, Ryerson","54 43.657162 Name: Latitude, dtype: float64","54 -79.378937 Name: Longitude, dtype: float64",0,Clothing Store,Coffee Shop,Cosmetics Shop,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Café,Restaurant,Bubble Tea Shop,Diner
3,M5C,Downtown Toronto,St. James Town,"55 43.651494 Name: Latitude, dtype: float64","55 -79.375418 Name: Longitude, dtype: float64",0,Café,Coffee Shop,Cocktail Bar,Restaurant,Gastropub,American Restaurant,Italian Restaurant,Gym,Department Store,Clothing Store
4,M4E,East Toronto,The Beaches,"37 43.676357 Name: Latitude, dtype: float64","37 -79.293031 Name: Longitude, dtype: float64",0,Trail,Pub,Health Food Store,Women's Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Cuban Restaurant


In [19]:
Toronto_merged.shape

(39, 16)